In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ACE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('angiotensin converting enzyme', 3262),
 ('adverse childhood experiences', 42),
 ('angiotensin converting enzyme gene', 16),
 ('angiotensinconverting enzyme', 15),
 ('acetate', 9),
 ('allele coupled exchange', 9),
 ('acetamiprid', 9),
 ('acesulfame', 7),
 ('acetylcholinesterase', 6),
 ('enzyme activity', 6),
 ('acetaldehyde', 5),
 ('action sentence compatibility effects', 5),
 ('agonist induced calcium entry', 5),
 ('anterior chamber of the eye', 5),
 ('agonist induced ca2 + entry', 5),
 ('acupoint catgut embedding', 4),
 ('antegrade continence enemas', 4),
 ('aqueous cinnamon extract', 4),
 ('angiotensin converting en zyme', 4),
 ('angiotensin converting enzyme inhibitors', 4),
 ('angiotensin converting enzyme 1', 4),
 ('anthocephalus cadmaba ethanol extract', 4),
 ('ac', 4),
 ('ace', 3),
 ('aceclofenac', 3),
 ('affinity capillaris electrophoresis', 3),
 ('arm crank ergometer', 3),
 ('anticipatory contrast effects', 3),
 ('ac extract', 3),
 ('angiotensin conversion enzyme', 3),
 ('a

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 03:15:34] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'a camphorata extract': 'ungrounded',
  'abundance based coverage estimation': 'ungrounded',
  'ac': 'ungrounded',
  'ac extract': 'ungrounded',
  'ace': 'ungrounded',
  'aceclofenac': 'CHEBI:CHEBI:31159',
  'acesulfame': 'CHEBI:CHEBI:83501',
  'acetaldehyde': 'CHEBI:CHEBI:15343',
  'acetamiprid': 'CHEBI:CHEBI:39163',
  'acetate': 'CHEBI:CHEBI:30089',
  'acetylcholinesterase': 'HGNC:12846',
  'action sentence compatibility effects': 'ungrounded',
  'acupoint catgut embedding': 'ungrounded',
  'acute care for elders': 'ungrounded',
  'adverse childhood experiences': 'MESH:D000076783',
  'affinity capillaris electrophoresis': 'ungrounded',
  'agonist induced ca2 + entry': 'ungrounded',
  'agonist induced calcium entry': 'ungrounded',
  'allele coupled exchange': 'ungrounded',
  'amygdaloid central nucleus': 'MESH:D002529',
  'angiotensin con verting enzyme': 'HGNC:2707',
  'angiotensin conversion enzyme': 'HGNC:2707',
  'angiotensin converting en zyme': 'HGNC:2707',
  'angiotensin conv

In [9]:
grounding_map, names, pos_labels = [{'a camphorata extract': 'ungrounded',
  'abundance based coverage estimation': 'ungrounded',
  'ac': 'ungrounded',
  'ac extract': 'ungrounded',
  'ace': 'ungrounded',
  'aceclofenac': 'CHEBI:CHEBI:31159',
  'acesulfame': 'CHEBI:CHEBI:83501',
  'acetaldehyde': 'CHEBI:CHEBI:15343',
  'acetamiprid': 'CHEBI:CHEBI:39163',
  'acetate': 'CHEBI:CHEBI:30089',
  'acetylcholinesterase': 'HGNC:12846',
  'action sentence compatibility effects': 'ungrounded',
  'acupoint catgut embedding': 'ungrounded',
  'acute care for elders': 'ungrounded',
  'adverse childhood experiences': 'MESH:D000076783',
  'affinity capillaris electrophoresis': 'ungrounded',
  'agonist induced ca2 + entry': 'ungrounded',
  'agonist induced calcium entry': 'ungrounded',
  'allele coupled exchange': 'ungrounded',
  'amygdaloid central nucleus': 'MESH:D002529',
  'angiotensin con verting enzyme': 'HGNC:2707',
  'angiotensin conversion enzyme': 'HGNC:2707',
  'angiotensin converting en zyme': 'HGNC:2707',
  'angiotensin converting enzyme': 'HGNC:2707',
  'angiotensin converting enzyme 1': 'HGNC:2707',
  'angiotensin converting enzyme gene': 'HGNC:2707',
  'angiotensin converting enzyme i': 'HGNC:2707',
  'angiotensin converting enzyme inhibitors': 'CHEBI:CHEBI:35457',
  'angiotensinconverting enzyme': 'HGNC:2707',
  'antegrade continence enemas': 'ungrounded',
  'anterior chamber of the eye': 'ungrounded',
  'anthocephalus cadmaba ethanol extract': 'ungrounded',
  'anticipatory contrast effects': 'ungrounded',
  'aqueous cinnamon extract': 'ungrounded',
  'arm crank ergometer': 'ungrounded',
  'central amygdaloid nucleus': 'MESH:D066274',
  'enzyme activity': 'GO:GO:0003824'},
 {'CHEBI:CHEBI:31159': 'aceclofenac',
  'CHEBI:CHEBI:83501': 'acesulfame',
  'CHEBI:CHEBI:15343': 'acetaldehyde',
  'CHEBI:CHEBI:39163': 'acetamiprid',
  'CHEBI:CHEBI:30089': 'acetate',
  'HGNC:12846': 'ACHE',
  'MESH:D000076783': 'Adverse Childhood Experiences',
  'MESH:D002529': 'Cerebellar Nuclei',
  'HGNC:2707': 'ACE',
  'CHEBI:CHEBI:35457': 'EC 3.4.15.1 (peptidyl-dipeptidase A) inhibitor',
  'MESH:D066274': 'Central Amygdaloid Nucleus',
  'GO:GO:0003824': 'catalytic activity'},
 ['HGNC:2707', 'MESH:D000076783']]

In [10]:
excluded_longforms = []

In [11]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [13]:
additional_entities = {}

In [14]:
unambiguous_agent_texts = {}

In [15]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update(additional_entities)
pos_labels.extend(additional_entities.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-02 03:23:30] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-02 03:24:22] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9802739223065566 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'HGNC:2707': 2647,
  'MESH:D066274': 3,
  'GO:GO:0003824': 6,
  'HGNC:12846': 5,
  'ungrounded': 44,
  'CHEBI:CHEBI:15343': 4,
  'MESH:D002529': 3,
  'CHEBI:CHEBI:35457': 4,
  'CHEBI:CHEBI:30089': 7,
  'CHEBI:CHEBI:31159': 3,
  'MESH:D000076783': 31,
  'CHEBI:CHEBI:39163': 7,
  'CHEBI:CHEBI:83501': 5},
 'f1': {'mean': 0.980274, 'std': 0.002623},
 'precision': {'mean': 0.969181, 'std': 0.001597},
 'recall': {'mean': 0.993281, 'std': 0.002789},
 'MESH:D066274': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.396737, 'std': 0.054055},
  'pr': {'mean': 0.272222, 'std': 0.050918},
  'rc': {'mean': 0.766667, 'std': 0.122474}},
 'CHEBI:CHEBI:31159': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0003824': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for ACE

Produces the disambiguations:
	ACE*	HGNC:2707
	ACHE	HGNC:12846
	Adverse Childhood Experiences*	MESH:D000076783
	Central Amygdaloid Nucleus	MESH:D066274
	Cerebellar Nuclei	MESH:D002529
	EC 3.4.15.1 (peptidyl-dipeptidase A) inhibitor	CHEBI:CHEBI:35457
	aceclofenac	CHEBI:CHEBI:31159
	acesulfame	CHEBI:CHEBI:83501
	acetaldehyde	CHEBI:CHEBI:15343
	acetamiprid	CHEBI:CHEBI:39163
	acetate	CHEBI:CHEBI:30089
	catalytic activity	GO:GO:0003824

Class level metrics:
--------------------
Grounding                                     	Count	F1     
                                           ACE*	2647	0.98306
                                    Ungrounded	  44	0.39674
                 Adverse Childhood Experiences*	  31	0.74242
                                       acetate	   7	    0.0
                                   acetamiprid	   7	0.33333
                            catalytic activity	   6	    0.0
                                          ACHE	   5	    0.0
         

In [24]:
model_to_s3(disamb)